<a href="https://colab.research.google.com/github/RamadanCRaji/student-loan-optimizer/blob/main/Ramadan_Raji_730_Application_Project_final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Student Loan Repayment Optimizer
### A Mixed Integer Programming Approach to Minimize Interest Payments

**Author:** Ramadan Raji  
**Program:** MS Business Analytics, UW-Madison School of Business  
**Course:** GENBUS 730 - Prescriptive Analytics  
**Date:** December 2025

---

## Why I Built This

I have student loans. Six of them, actually, totaling over $33,000 with interest rates ranging from 3.76% to 7.94%. Like most students, I was making payments without really knowing if I was doing it the "right" way. Should I pay them all equally? Attack the highest balance first? Focus on the highest interest rate?

Then I took my Prescriptive Analytics class and learned about optimization modeling. We shifted from a "data first" approach to a "model first, then data" approach. That's when it clicked: I could use these new skills to solve my own real problem.

So I built this optimizer. It takes my actual loan data, my budget constraints, and figures out the mathematically optimal way to pay off my debt while minimizing total interest paid.

**The result?** By following the optimal strategy instead of splitting payments equally, I save **$1,279** in interest. That's **20.7% less interest** with the exact same budget.

---

## What This Notebook Does

This notebook builds a Mixed Integer Programming (MIP) model that:

- Takes in loan balances, interest rates, and budget constraints
- Accounts for subsidized vs unsubsidized loans (subsidized loans do not accrue interest while in school)
- Handles two budget phases: 400/month while in school, $800/month after graduation
- Outputs a quarter-by-quarter payment schedule telling you exactly how much to pay each loan

---

## Technical Stack

- **Python** for programming
- **Pyomo** for optimization modeling
- **CBC Solver** (via IDAES) for solving the MIP
- **Google Colab** for running the notebook

---

## How to Use This Notebook

1. Run Cell 1 to install dependencies
2. Modify Cell 3 with your own loan data (balances, rates, subsidized status)
3. Adjust your budget in Cell 3 (M1 = in school, M2 = after graduation)
4. Run all cells
5. Check the output for your personalized payment plan

---




#**Application Context:**
> MIP optimization model that determines the optimal payment strategy across multiple student loans with different interest rates and subsidy statuses. Minimizes total interest paid over a 5-year horizon subject to budget constraints. Built with Pyomo and solved using CBC. Saves ~$1,279 (20.7%) compared to a naive equal-split strategy.



In [34]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *
from tabulate import tabulate
!pip install tabulate

In [35]:

# =============================================================================
# CELL 2: CREATE MODEL + SETS
# =============================================================================

model = ConcreteModel(name="Student_Loan_Repayment_Optimizer")

# Sets
model.I = RangeSet(1, 6)   # Number of loans I am dealing with is -> 6 loans
model.T = RangeSet(1, 20)  # when i plan to pay the loans of in terms of quaters 20 quarters (5 years)

print("Model created with 6 loans over 20 quarters")



Model created with 6 loans over 20 quarters


In [36]:

# =============================================================================
# CELL 3: PARAMETERS (YOUR LOAN DATA)
# =============================================================================

# Starting balances
starting_balance = {
    1: 3522.00,   # 1-01 Subsidized (4.29%)
    2: 3518.00,   # 1-03 Subsidized (3.76%)
    3: 2054.00,   # 1-05 Subsidized (4.45%)
    4: 1204.00,   # 1-06 Subsidized (5.05%)
    5: 12528.00,  # 1-07 Unsubsidized (7.94%) - ACCRUING NOW
    6: 10295.00,  # 1-08 Unsubsidized (7.94%) - ACCRUING NOW
}
model.B = Param(model.I, initialize=starting_balance) #pyomo matches the slots to the appropriate loans

# Quarterly interest rates (annual / 4) for each loans will be designated as r with r[1] refrencing interest loan 1
quarterly_rates = {
    1: 0.0429 / 4,  # 4.29% annual
    2: 0.0376 / 4,  # 3.76% annual
    3: 0.0445 / 4,  # 4.45% annual
    4: 0.0505 / 4,  # 5.05% annual
    5: 0.0794 / 4,  # 7.94% annual
    6: 0.0794 / 4,  # 7.94% annual
}
model.r = Param(model.I, initialize=quarterly_rates) #param takes in arguments index, now r[i] gives me the quaterly rate  for loan[i]


# Subsidized indicator (1 = subsidized, no interest while in school)
subsidized = {1: 1, 2: 1, 3: 1, 4: 1, 5: 0, 6: 0}
model.sub = Param(model.I, initialize=subsidized)

# Phase parameters- not indexed items since this are individual constraints
model.T1 = Param(initialize=2)      # Last quarter of Phase 1 (in school), so phase 1 of payment will end after Q2
model.M1 = Param(initialize=1200)   # $400/month × 3 = $1,200/quarter budget (in school)
model.M2 = Param(initialize=2400)   # $800/month × 3 = $2,400/quarter budget (after grad)

#Items below are teo
print(" Parameters loaded")
print(f"  Total Starting Debt: ${sum(starting_balance.values()):,.2f}")
print(f"  Phase 1 Budget: ${value(model.M1)/3:.0f}/month (while in school)") # we do not want any floats after the division happens
print(f"  Phase 2 Budget: ${value(model.M2)/3:.0f}/month (after graduation)")




 Parameters loaded
  Total Starting Debt: $33,121.00
  Phase 1 Budget: $400/month (while in school)
  Phase 2 Budget: $800/month (after graduation)


**Above are the establised numbers and loan value I have can view via my fafsa account**

In [37]:
# =============================================================================
# CELL 4: DECISION VARIABLES
# =============================================================================

# x[i,t] = dollars paid toward loan i in quarter t
model.x = Var(model.I, model.T, domain=NonNegativeReals, doc="Payment amount")

# y[i,t] = 1 if loan i is fully paid off by end of quarter t (BINARY - makes it MIP)
model.y = Var(model.I, model.T, domain=Binary, doc="Payoff indicator")

# Auxiliary variables- this tells me the balance of laon i at the end of quater t
model.L = Var(model.I, model.T, domain=NonNegativeReals, doc="Loan balance")
model.Interest = Var(model.I, model.T, domain=NonNegativeReals, doc="Interest accrued") #interest of loan i in quater t

print("Variables created")
print(f"  Decision: x[i,t] = payment to loan i in quarter t (continuous)")
print(f"  Decision: y[i,t] = 1 if loan i paid off by quarter t (binary)")
print(f"  Auxiliary: L[i,t] = balance, Interest[i,t] = interest accrued")

# the payoff could not be decison variables, I need to think about that for future iteration

Variables created
  Decision: x[i,t] = payment to loan i in quarter t (continuous)
  Decision: y[i,t] = 1 if loan i paid off by quarter t (binary)
  Auxiliary: L[i,t] = balance, Interest[i,t] = interest accrued


In [38]:

# =============================================================================
# CELL 5: OBJECTIVE FUNCTION
# =============================================================================

def objective_rule(model):
    """Minimize total interest paid across all loans and quarters"""
    return sum(model.Interest[i, t] for i in model.I for t in model.T)

model.obj = Objective(rule=objective_rule, sense=minimize)

print("Objective: Minimize total interest paid")




Objective: Minimize total interest paid


In [39]:
# =============================================================================
# CELL 6: CONSTRAINTS
# =============================================================================

# Constraint 1: Interest Calculation
def interest_rule(model, i, t):
    """Subsidized loans: no interest while in school. Unsubsidized: always accrues."""
    if t == 1:
        if model.sub[i] == 1 and t <= model.T1:
            return model.Interest[i, t] == 0
        else:
            return model.Interest[i, t] == model.r[i] * model.B[i]
    else:
        if model.sub[i] == 1 and t <= model.T1:
            return model.Interest[i, t] == 0
        else:
            return model.Interest[i, t] == model.r[i] * model.L[i, t-1]

model.interest_calc = Constraint(model.I, model.T, rule=interest_rule)

# Constraint 2: Balance Evolution
def balance_rule(model, i, t):
    """Balance = previous balance + interest - payment"""
    if t == 1:
        return model.L[i, t] == model.B[i] + model.Interest[i, t] - model.x[i, t]
    else:
        return model.L[i, t] == model.L[i, t-1] + model.Interest[i, t] - model.x[i, t]

model.balance_evolution = Constraint(model.I, model.T, rule=balance_rule)

# Constraint 3: Budget Constraint
def budget_rule(model, t):
    """Total payments cannot exceed budget (different for Phase 1 vs 2)"""
    if t <= model.T1:
        return sum(model.x[i, t] for i in model.I) <= model.M1
    else:
        return sum(model.x[i, t] for i in model.I) <= model.M2

model.budget_constraint = Constraint(model.T, rule=budget_rule)

# Constraint 4: Can't Pay More Than You Owe
def max_payment_rule(model, i, t):
    """Payment cannot exceed current balance + interest"""
    if t == 1:
        return model.x[i, t] <= model.B[i] + model.Interest[i, t]
    else:
        return model.x[i, t] <= model.L[i, t-1] + model.Interest[i, t]

model.max_payment = Constraint(model.I, model.T, rule=max_payment_rule)

# Constraint 5: Payoff Tracking (THIS IS THE MIP COMPONENT)
def payoff_tracking_rule(model, i, t):
    """If y[i,t]=1 (paid off), balance must be 0. Uses Big-M linearization."""
    BigM = 50000
    return model.L[i, t] <= BigM * (1 - model.y[i, t])

model.payoff_tracking = Constraint(model.I, model.T, rule=payoff_tracking_rule)

# Constraint 6: Payoff Is Permanent
def payoff_permanent_rule(model, i, t):
    """Once paid off, stays paid off"""
    if t == 1:
        return Constraint.Skip
    else:
        return model.y[i, t] >= model.y[i, t-1]

model.payoff_permanent = Constraint(model.I, model.T, rule=payoff_permanent_rule)

# Constraint 7: Final Balance Must Be Zero
def final_payoff_rule(model, i):
    """All loans must be paid off by end of horizon"""
    return model.L[i, 20] == 0

model.final_payoff = Constraint(model.I, rule=final_payoff_rule)

# Constraint 8: Minimum Payment Required (once in repayment)
def min_payment_rule(model, i, t):
    """Federal loans require ~1% of balance/month = 3%/quarter. Linearized with Big-M."""
    if t <= value(model.T1):
        return Constraint.Skip
    if t == 1:
        return Constraint.Skip
    min_pct = 0.03
    BigM = 50000
    return model.x[i, t] >= min_pct * model.L[i, t-1] - BigM * model.y[i, t-1]

model.min_payment = Constraint(model.I, model.T, rule=min_payment_rule)

# Count constraints
num_constraints = sum(len(c) for c in model.component_objects(Constraint, active=True))
print(f"✓ Constraints created: {num_constraints} total")
print(f"  1. Interest calculation: 120")
print(f"  2. Balance evolution: 120")
print(f"  3. Budget constraint: 20")
print(f"  4. Max payment: 120")
print(f"  5. Payoff tracking: 120")
print(f"  6. Payoff permanent: 114")
print(f"  7. Final payoff: 6")
print(f"  8. Minimum payment: 108")




✓ Constraints created: 728 total
  1. Interest calculation: 120
  2. Balance evolution: 120
  3. Budget constraint: 20
  4. Max payment: 120
  5. Payoff tracking: 120
  6. Payoff permanent: 114
  7. Final payoff: 6
  8. Minimum payment: 108


In [40]:
# =============================================================================
# CELL 7: SOLVE
# =============================================================================

print("=" * 60)
print("SOLVING MODEL...")
print("=" * 60)

solver = SolverFactory('cbc', executable='./bin/cbc')
solver.options['ratioGap'] = 0.01   # Stop at 1% of optimal
solver.options['seconds'] = 120     # Max 2 minutes

results = solver.solve(model, tee=False)

print(f"\nSolver Status: {results.solver.termination_condition}")


SOLVING MODEL...

Solver Status: optimal


In [41]:
# =============================================================================
# CELL 8: DISPLAY RESULTS
# =============================================================================

from tabulate import tabulate

# ANSI color codes
RED = "\033[91m"
GREEN = "\033[92m"
MAGENTA = "\033[95m"   # Replaces yellow
BLUE = "\033[94m"      # Replaces cyan
BOLD = "\033[1m"
RESET = "\033[0m"

if results.solver.termination_condition in [TerminationCondition.optimal, TerminationCondition.feasible]:
    print(f"\n{BOLD}{'=' * 60}")
    print("OPTIMIZATION RESULTS")
    print(f"{'=' * 60}{RESET}")

    # Calculate totals
    total_interest = sum(value(model.Interest[i, t]) for i in model.I for t in model.T)
    total_payments = sum(value(model.x[i, t]) for i in model.I for t in model.T)

    print(f"\n{GREEN}✓ Solution found!{RESET}")
    print(f"\n{BOLD}TOTAL INTEREST YOU'LL PAY:{RESET} {RED}${total_interest:,.2f}{RESET}")
    print(f"{BOLD}TOTAL PAYMENTS:{RESET} ${total_payments:,.2f}")
    print(f"{BOLD}TOTAL STARTING DEBT:{RESET} ${sum(starting_balance.values()):,.2f}")

    # Loan names for display
    loan_names = {
        1: "1-01 Sub (4.29%)",
        2: "1-03 Sub (3.76%)",
        3: "1-05 Sub (4.45%)",
        4: "1-06 Sub (5.05%)",
        5: "1-07 Unsub (7.94%)",
        6: "1-08 Unsub (7.94%)",
    }

    # Quarter to date mapping
    quarter_to_date = {
        1: "Jan-Mar 2026", 2: "Apr-Jun 2026",
        3: "Jul-Sep 2026", 4: "Oct-Dec 2026",
        5: "Jan-Mar 2027", 6: "Apr-Jun 2027",
        7: "Jul-Sep 2027", 8: "Oct-Dec 2027",
        9: "Jan-Mar 2028", 10: "Apr-Jun 2028",
        11: "Jul-Sep 2028", 12: "Oct-Dec 2028",
        13: "Jan-Mar 2029", 14: "Apr-Jun 2029",
        15: "Jul-Sep 2029", 16: "Oct-Dec 2029",
        17: "Jan-Mar 2030", 18: "Apr-Jun 2030",
        19: "Jul-Sep 2030", 20: "Oct-Dec 2030",
    }

    # Payoff Timeline
    print(f"\n{BOLD}{'-' * 60}")
    print("PAYOFF TIMELINE:")
    print(f"{'-' * 60}{RESET}")

    for i in model.I:
        for t in model.T:
            if value(model.y[i, t]) > 0.5:
                quarter = t
                dates = quarter_to_date[t]
                if i <= 4:  # Subsidized = green
                    print(f"  {GREEN}{loan_names[i]}: Paid off Q{quarter} ({dates}){RESET}")
                else:  # Unsubsidized = red
                    print(f"  {RED}{loan_names[i]}: Paid off Q{quarter} ({dates}){RESET}")
                break
        else:
            final_balance = value(model.L[i, 20])
            print(f"  {loan_names[i]}: ${final_balance:,.2f} remaining")

    # Quarterly Payment Schedule with tabulate
    print(f"\n{BOLD}QUARTERLY PAYMENT SCHEDULE:{RESET}")

    table_data = []
    for t in model.T:
        payments = [value(model.x[i, t]) for i in model.I]
        total = sum(payments)
        if total > 0.01:
            row = [
                f"{BLUE}Q{t}{RESET}",
                f"{quarter_to_date[t]}",
                f"{GREEN}${payments[0]:.0f}{RESET}",
                f"{GREEN}${payments[1]:.0f}{RESET}",
                f"{GREEN}${payments[2]:.0f}{RESET}",
                f"{GREEN}${payments[3]:.0f}{RESET}",
                f"{RED}${payments[4]:.0f}{RESET}",
                f"{RED}${payments[5]:.0f}{RESET}",
                f"{BOLD}${total:.0f}{RESET}"
            ]
            table_data.append(row)

    headers = [
        f"{BOLD}Qtr{RESET}",
        f"{BOLD}Dates{RESET}",
        f"{GREEN}Loan1\n(1-01){RESET}",
        f"{GREEN}Loan2\n(1-03){RESET}",
        f"{GREEN}Loan3\n(1-05){RESET}",
        f"{GREEN}Loan4\n(1-06){RESET}",
        f"{RED}Loan5\n(1-07){RESET}",
        f"{RED}Loan6\n(1-08){RESET}",
        f"{BOLD}Total{RESET}"
    ]

    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    # Interest Breakdown
    print(f"\n{BOLD}{'-' * 60}")
    print("INTEREST PAID BY LOAN:")
    print(f"{'-' * 60}{RESET}")

    for i in model.I:
        interest = sum(value(model.Interest[i, t]) for t in model.T)
        if i <= 4:  # Subsidized = green
            print(f"  {GREEN}{loan_names[i]}: ${interest:,.2f}{RESET}")
        else:  # Unsubsidized = red
            print(f"  {RED}{loan_names[i]}: ${interest:,.2f}{RESET}")

else:
    print(f"\n{RED}✗ Solver failed: {results.solver.termination_condition}{RESET}")


OPTIMIZATION RESULTS

✓ Solution found!

TOTAL INTEREST YOU'LL PAY: $4,912.98
TOTAL PAYMENTS: $38,033.98
TOTAL STARTING DEBT: $33,121.00

------------------------------------------------------------
PAYOFF TIMELINE:
------------------------------------------------------------
  1-01 Sub (4.29%): Paid off Q16 (Oct-Dec 2029)
  1-03 Sub (3.76%): Paid off Q18 (Apr-Jun 2030)
  1-05 Sub (4.45%): Paid off Q16 (Oct-Dec 2029)
  1-06 Sub (5.05%): Paid off Q14 (Apr-Jun 2029)
  1-07 Unsub (7.94%): Paid off Q20 (Oct-Dec 2030)
  1-08 Unsub (7.94%): Paid off Q20 (Oct-Dec 2030)

QUARTERLY PAYMENT SCHEDULE:
+-------+--------------+----------+----------+----------+----------+----------+----------+---------+
| Qtr   | Dates        | Loan1    | Loan2    | Loan3    | Loan4    | Loan5    | Loan6    | Total   |
|       |              | (1-01)   | (1-03)   | (1-05)   | (1-06)   | (1-07)   | (1-08)   |         |
+=======+==============+==========+==========+==========+==========+==========+==========+========

In [42]:
# =============================================================================
# CELL 9: EXECUTIVE SUMMARY & COMPARISON
# =============================================================================

if results.solver.termination_condition in [TerminationCondition.optimal, TerminationCondition.feasible]:

    # Find last quarter with payments
    last_payment_quarter = 0
    for t in model.T:
        if sum(value(model.x[i, t]) for i in model.I) > 0.01:
            last_payment_quarter = t

    quarter_to_date = {
        1: "Jan 2026 - Mar 2026", 2: "Apr 2026 - Jun 2026",
        3: "Jul 2026 - Sep 2026", 4: "Oct 2026 - Dec 2026",
        5: "Jan 2027 - Mar 2027", 6: "Apr 2027 - Jun 2027",
        7: "Jul 2027 - Sep 2027", 8: "Oct 2027 - Dec 2027",
        9: "Jan 2028 - Mar 2028", 10: "Apr 2028 - Jun 2028",
        11: "Jul 2028 - Sep 2028", 12: "Oct 2028 - Dec 2028",
        13: "Jan 2029 - Mar 2029", 14: "Apr 2029 - Jun 2029",
        15: "Jul 2029 - Sep 2029", 16: "Oct 2029 - Dec 2029",
        17: "Jan 2030 - Mar 2030", 18: "Apr 2030 - Jun 2030",
        19: "Jul 2030 - Sep 2030", 20: "Oct 2030 - Dec 2030",
    }

    print(f"\n{BOLD}{BLUE}{'=' * 60}")
    print("EXECUTIVE SUMMARY: YOUR ACTION PLAN")
    print(f"{'=' * 60}{RESET}")

    print(f"\n→ {BOLD}DEBT-FREE BY:{RESET} {GREEN}Q{last_payment_quarter} ({quarter_to_date.get(last_payment_quarter, 'N/A')}){RESET}")
    print(f"→ {BOLD}TOTAL INTEREST PAID:{RESET} {RED}${total_interest:,.2f}{RESET}")
    print(f"→ {BOLD}TOTAL OUT OF POCKET:{RESET} ${total_payments:,.2f}")

    print(f"\n{BOLD}{'-' * 60}")
    print("SIMPLIFIED ACTION PLAN:")
    print(f"{'-' * 60}{RESET}")

    print(f"\n  {MAGENTA}{BOLD}PHASE 1 - WHILE IN SCHOOL ({quarter_to_date[1]} to {quarter_to_date[2]}):{RESET}")
    print(f"    • Pay $400/month → {RED}ALL to Loan 1-07 (7.94% Unsubsidized){RESET}")
    print(f"    • Why: This loan is accruing interest NOW. Subsidized loans are frozen.")

    print(f"\n  {MAGENTA}{BOLD}PHASE 2 - AFTER GRADUATION ({quarter_to_date[3]} onwards):{RESET}")
    print(f"    • Pay $800/month → {RED}Focus on BOTH 7.94% Unsubsidized loans{RESET}")
    print(f"    • Clear Loan 1-07 and 1-08 before touching subsidized loans")

    print(f"\n  {MAGENTA}{BOLD}PHASE 3 - FINAL STRETCH:{RESET}")
    print(f"    • Once unsubsidized loans are gone, {GREEN}clean up the 4 subsidized loans{RESET}")
    print(f"    • These have lower rates (3.76% - 5.05%), less urgent")

    # Comparison vs Naive with tabulate
    print(f"\n{BOLD}{'-' * 60}")
    print("COMPARISON: OPTIMAL vs EQUAL SPLIT STRATEGY")
    print(f"{'-' * 60}{RESET}")

    naive_balances = {i: starting_balance[i] for i in range(1, 7)}
    naive_interest_total = 0

    for t in range(1, 21):
        if t <= 2:
            quarterly_budget = 1200
        else:
            quarterly_budget = 2400

        active_count = sum(1 for i in range(1, 7) if naive_balances[i] > 0)
        if active_count == 0:
            break

        payment_per_loan = quarterly_budget / active_count

        for i in range(1, 7):
            if naive_balances[i] > 0:
                if subsidized[i] == 1 and t <= 2:
                    interest = 0
                else:
                    interest = naive_balances[i] * quarterly_rates[i]

                naive_interest_total += interest
                naive_balances[i] += interest - min(payment_per_loan, naive_balances[i] + interest)
                if naive_balances[i] < 0.01:
                    naive_balances[i] = 0

    savings = naive_interest_total - total_interest

    comparison_data = [
        [f"{GREEN}{BOLD}OPTIMAL (This Model){RESET}", f"{GREEN}${total_interest:,.2f}{RESET}", f"${total_payments:,.2f}"],
        [f"{RED}Equal Split (Naive){RESET}", f"{RED}${naive_interest_total:,.2f}{RESET}", f"${sum(starting_balance.values()) + naive_interest_total:,.2f}"]
    ]
    comparison_headers = [f"{BOLD}Strategy{RESET}", f"{BOLD}Total Interest{RESET}", f"{BOLD}Total Paid{RESET}"]

    print(tabulate(comparison_data, headers=comparison_headers, tablefmt="grid"))

    print(f"\n  → {BOLD}YOU SAVE: {GREEN}${savings:,.2f}{RESET}{BOLD} by following the optimal strategy!{RESET}")
    print(f"  → That's {GREEN}{savings/naive_interest_total*100:.1f}% less interest{RESET} paid.")

    print(f"\n{BOLD}{BLUE}{'=' * 60}")
    print("END OF REPORT")
    print(f"{'=' * 60}{RESET}")


EXECUTIVE SUMMARY: YOUR ACTION PLAN

→ DEBT-FREE BY: Q17 (Jan 2030 - Mar 2030)
→ TOTAL INTEREST PAID: $4,912.98
→ TOTAL OUT OF POCKET: $38,033.98

------------------------------------------------------------
SIMPLIFIED ACTION PLAN:
------------------------------------------------------------

  PHASE 1 - WHILE IN SCHOOL (Jan 2026 - Mar 2026 to Apr 2026 - Jun 2026):
    • Pay $400/month → ALL to Loan 1-07 (7.94% Unsubsidized)
    • Why: This loan is accruing interest NOW. Subsidized loans are frozen.

  PHASE 2 - AFTER GRADUATION (Jul 2026 - Sep 2026 onwards):
    • Pay $800/month → Focus on BOTH 7.94% Unsubsidized loans
    • Clear Loan 1-07 and 1-08 before touching subsidized loans

  PHASE 3 - FINAL STRETCH:
    • Once unsubsidized loans are gone, clean up the 4 subsidized loans
    • These have lower rates (3.76% - 5.05%), less urgent

------------------------------------------------------------
COMPARISON: OPTIMAL vs EQUAL SPLIT STRATEGY
-------------------------------------------

---

# Summary and Reflections

## What I Accomplished

I built a Mixed Integer Programming model that optimizes student loan repayment. Using my real loan data from Federal Student Aid, the model determined the optimal payment strategy across 20 quarters (5 years).

**Key Results:**
- Total interest paid: **$4,913** (optimal) vs **$6,192** (naive equal split)
- Total savings: **$1,279** (20.7% less interest)
- Debt-free by: **Q17 (January - March 2030)**

The model confirmed what finance experts often recommend: attack high-interest debt first. But it went further by accounting for the fact that my subsidized loans do not accrue interest while I am in school, so every dollar should go toward the unsubsidized loans during that time.

---

## What I Learned

This project taught me that optimization is not just an academic exercise. It solves real problems. The shift from "data first" to "model first, then data" changed how I think about problem-solving.

I also learned that the math behind these models is powerful when combined with domain knowledge. Understanding how federal student loans work (subsidized vs unsubsidized, grace periods, minimum payments) was just as important as knowing how to write constraints in Pyomo.

---

## Challenges I Faced

1. **Scalability:** My first model used monthly periods (60 time steps) and took too long to solve. Switching to quarterly periods (20 time steps) solved in under 1 second.

2. **Nonlinear Constraints:** The minimum payment rule created a nonlinear term. I used Big-M linearization to convert it into a linear constraint.

3. **Solver Setup:** Getting CBC to work in Google Colab required using IDAES extensions to access the solver executable.

---

## Future Improvements

- Add income growth scenarios (budget increases over time as salary grows)
- Model refinancing decisions (lower rate vs losing federal protections)
- Build a web app so anyone can input their loans and get an optimized plan
- Add PSLF (Public Service Loan Forgiveness) modeling for public sector workers

---

## Final Thoughts

If you have student loans, you do not have to guess at the best repayment strategy. With some basic optimization modeling, you can find the mathematically optimal approach for your specific situation.

Feel free to fork this notebook and plug in your own loan data. If you have questions, reach out!

---

**Tools Used:** Python, Pyomo, CBC Solver, Google Colab  
**Course:** GENBUS 730 - Prescriptive Analytics  
**University:** UW-Madison School of Business